# Dynamically Selected Tools

## Load MCP Tools

In [ ]:
import json
import os

def load_mcp_servers(config_path):
    """
    Load MCP server definitions from a JSON config file.
    Expects a top-level 'mcpServers' dict in the config.
    """
    if not os.path.exists(config_path):
        raise FileNotFoundError(f"Config file not found: {config_path}")
    with open(config_path, "r") as f:
        config = json.load(f)
    servers = config.get("mcpServers", {})
    # Optionally add default transports if missing
    for name, server in servers.items():
        if "command" in server and "transport" not in server:
            server["transport"] = "stdio"
        if "url" in server and "transport" not in server:
            server["transport"] = "streamable_http"
    return servers

mcp_servers = load_mcp_servers("./mcp_config.json")
mcp_servers

In [ ]:
from langchain_mcp_adapters.client import MultiServerMCPClient

client = MultiServerMCPClient(mcp_servers)
mcp_tools = await client.get_tools()

print(f"Loaded {len(mcp_tools)} tools.\n")
print(f"Example of a tool:\n{mcp_tools[0]}")

## Single ReAct Agent with all Tools

In [ ]:

from langgraph.prebuilt import create_react_agent
from IPython.display import Image, display
from tools import draw_mermaid_png

GITHUB_AGENT_SYSTEM_MESSAGE = """
You are a GitHub Assistant that helps users manage their GitHub repositories and workflows.

You can help with:
- Repository management (create, fork, browse files)
- Issues and pull requests (create, review, merge)
- Code operations (search, commit, push changes)
- GitHub Actions workflows (run, monitor, debug)
- Notifications and alerts

Use the appropriate GitHub tools based on user requests. 
For complex tasks, break them down into steps and explain what you're doing along the way.

When a user needs help with GitHub, they should simply describe what they want to accomplish, 
and you'll guide them through the process using the available tools.
"""

github_agent = create_react_agent(
    model="openai:gpt-4o-mini",
    tools=mcp_tools,
    prompt=GITHUB_AGENT_SYSTEM_MESSAGE
)

draw_mermaid_png(github_agent)

## Reducing number of tools

1) decide what tools can be usefull based on user's request

2) provide model dynamically with reduced set of tools

### Reducing number of tools

In [ ]:
from langchain.chat_models import init_chat_model
from textwrap import dedent
from pydantic import BaseModel, Field
from typing import List

model = init_chat_model("openai:gpt-4o-mini", temperature=0)

# Define structured schema for tool selection
class ToolSelection(BaseModel):
    tools: List[str] = Field(description="A list of tool names that are most relevant to the user's query")

# Initialize structured model
structured_model = model.with_structured_output(ToolSelection)

def select_tools_via_llm(query: str, all_tools: list, max_tools: int = 7) -> List[str]:
    """
    Uses an LLM to select the most relevant tool names based on the user query.
    Returns a list of tool names (strings).
    """
    tools_text = "\n".join(
        f"- {tool.name}: {tool.description}" for tool in all_tools
    )

    prompt = dedent(f"""
        You are a tool selector agent. Your job is to choose the most relevant tools based on the user's query.

        User query:
        \"\"\"{query}\"\"\"

        Available tools:
        {tools_text}

        Return only the tools that are most relevant to the user query. Do not include tools that are unrelated or overly generic.
        Select up to {max_tools} tools.
    """)

    return structured_model.invoke(prompt).tools

In [ ]:
prompt = dedent("""
Can you check my account on GitHub and look at my recent work on the langgraph-advanced repo? 
I want to understand what I've been working on lately.
So collect information about all my recent activity and provide a brief overview in natural human language 
about my recent work.
""")

select_tools_via_llm(prompt, mcp_tools)

### Runtime context

In [ ]:
from dataclasses import dataclass
from langgraph.runtime import Runtime

@dataclass
class CustomContext:
    tools: List[str]  # tool names selected for this run


def configure_model(state, runtime: Runtime[CustomContext]):
    selected_tools = [
        tool for tool in mcp_tools
        if tool.name in runtime.context.tools
    ]
    return model.bind_tools(selected_tools)

### Create ReAct agent with dynamically selected Tools

In [ ]:

from langgraph.prebuilt import create_react_agent
from IPython.display import Image, display
from tools import draw_mermaid_png

GITHUB_AGENT_SYSTEM_MESSAGE = """
You are a GitHub Assistant that helps users manage their GitHub repositories and workflows.

You can help with:
- Repository management (create, fork, browse files)
- Issues and pull requests (create, review, merge)
- Code operations (search, commit, push changes)
- GitHub Actions workflows (run, monitor, debug)
- Notifications and alerts

Use the appropriate GitHub tools based on user requests. 
For complex tasks, break them down into steps and explain what you're doing along the way.

When a user needs help with GitHub, they should simply describe what they want to accomplish, 
and you'll guide them through the process using the available tools.
"""

github_agent = create_react_agent(
    model=configure_model,
    tools=mcp_tools,
    prompt=GITHUB_AGENT_SYSTEM_MESSAGE
)

draw_mermaid_png(github_agent)

### Testing

In [ ]:
from langchain_core.messages import HumanMessage
from textwrap import dedent

user_query = dedent("""
Can you check my account on GitHub and look at my recent work on the langgraph-advanced repo? 
I want to understand what I've been working on lately.
So collect information about all my recent activity and provide a brief overview in natural human language 
about my recent work.
""")

# Select relevant tools dynamically using LLM
selected_tool_names = select_tools_via_llm(user_query, mcp_tools)

# Prepare context with selected tools
context = CustomContext(tools=selected_tool_names)

# Run the agent graph with input and context
result = await github_agent.ainvoke(
    {"messages": [HumanMessage(content=user_query)]},
    context=context
)

for message in result['messages']:
    message.pretty_print()

## Reference Links

**1. LangGraph Prebuilt ReAct Agent (create_react_agent)**

https://langchain-ai.github.io/langgraph/reference/agents/#langgraph.prebuilt.chat_agent_executor.create_react_agent

→ API reference for the create_react_agent function, covering static/dynamic model selection, prompt customization, memory, hooks, and human-in-the-loop features.

**2. Dynamically Select Tools in LangGraph**

https://langchain-ai.github.io/langgraph/how-tos/tool-calling/#dynamically-select-tools

→ How-to guide for implementing dynamic tool selection in LangGraph agents, including runtime tool filtering, conditional tool availability, and context-aware tool routing.